In [1]:
import pandas as pd
import nlp
import numpy as np
import torch

In [2]:
datasets = {
    "cw-train": nlp.load_dataset("csv", name="cw-train", data_files="data/checkworthy/CT24_checkworthy_english_train.tsv", delimiter="\t"),
    "cw-dev": nlp.load_dataset("csv", name="cw-dev", data_files="data/checkworthy/CT24_checkworthy_english_dev.tsv", delimiter="\t"),
        }
    #"cw-dev-test": nlp.load_dataset("csv", name="cw-dev-test", data_files="data/checkworthy/CT24_checkworthy_english_dev-test.tsv", delimiter="\t"),

    #TODO: Add the other datasets including the other one about stance detection
    #TODO: Figure out which datasets is which, by that, I mean which one is training, which one is testing, etc. 

Using custom data configuration cw-train
Using custom data configuration cw-dev


In [3]:
for task_name, dataset in datasets.items():
    print(task_name)
    print(datasets[task_name]["train"][1])
    print()

cw-train
{'Sentence_id': 19099, 'Text': "Now, let's balance the budget and protect Medicare, Medicaid, education and the environment.", 'class_label': 'No'}

cw-dev
{'Sentence_id': 80, 'Text': "We're consuming 50 percent of the world's cocaine.", 'class_label': 'Yes'}



In [4]:
#https://colab.research.google.com/github/zphang/zphang.github.io/blob/master/files/notebooks/Multi_task_Training_with_Transformers_NLP.ipynb#scrollTo=aVX5hFlzmLka
import transformers
import torch.nn as nn
class MultiTask(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        super().__init__(transformers.PretrainedConfig())
        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def create(cls, model_name, model_type_dict, model_conf_dict):
        shared_encoder = None
        taskmodels_dict = {}

        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained(
                model_name,
                config=model_conf_dict[task_name]
            )

            if shared_encoder is None:
                shared_encoder = getattr(model, cls.get_encoder_attr_name(model))
            else:
                setattr(model, cls.get_encoder_attr_name(model), shared_encoder)
            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)
        
    @classmethod
    def get_encoder_attr_name(cls, model):
        model_class_name = model.__class__.__name__
        if model_class_name.startswith("Bert"):
            return "bert"
        elif model_class_name.startswith("Roberta"):
            return "roberta"

        else:
            raise KeyError(f"Add support for new model {model_class_name}")
        
    def forward(self, task_name, **kwargs):
        return self.taskmodels_dict[task_name](**kwargs)
#test = MultiTask(transformers.BertModel.from_pretrained("bert-base-uncased"), datasets)

In [5]:
model_name = "roberta-base"
#from transformers import BertTokenizer, BertForSequenceClassification
#"cw-train": transformers.
model_type_dict={
        "cw-train": transformers.AutoModelForSequenceClassification,
        "cw-dev": transformers.AutoModelForSequenceClassification,
    }
        #"test": transformers.AutoModelForTokenClassification

model_config_dict={
        "cw-train": transformers.AutoConfig.from_pretrained(model_name, num_labels=3),
        "cw-dev": transformers.AutoConfig.from_pretrained(model_name, num_labels=3),
    }
        #"test": transformers.AutoConfig.from_pretrained(model_name, num_labels=5)
#TODO: Add the other datasets in these two dictionaries. Should prolly use the AutoModelForTokenClassification for the other datasets
multitask_model = MultiTask.create(
    model_name,
    model_type_dict,
    model_config_dict
    
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
if model_name.startswith("roberta"):
    print(multitask_model.encoder.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["cw-train"].roberta.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["cw-dev"].roberta.embeddings.word_embeddings.weight.data_ptr())
else:
    print("Exercise for the reader: add a check for other model architectures =)")

139840211583040
139840211583040
139840211583040


In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [8]:
from sklearn import preprocessing
max_length = 128

def convert_cw_train_features(batch):
    inputs = batch['Text']
    features = tokenizer.batch_encode_plus(
        inputs,
        max_length=max_length,
        pad_to_max_length=True
    )
    le = preprocessing.LabelEncoder()
    targets = le.fit_transform(batch["class_label"])
    features["labels"] = targets
    #features["labels"] = batch["class_label"]
    return features

def convert_cw_dev_features(batch):
    inputs = batch['Text']
    features = tokenizer.batch_encode_plus(
        inputs,
        max_length=max_length,
        pad_to_max_length=True
    )
    le = preprocessing.LabelEncoder()
    targets = le.fit_transform(batch["class_label"])
    features["labels"] = targets
    #features["labels"] = batch["class_label"]
    return features

#TODO: Create funcs for the other datasets as well, do prolly have to do some finicky shit with TokenClassifications contra SequenceClassification

convert_func_dict = {
    "cw-train": convert_cw_train_features,
    "cw-dev": convert_cw_dev_features
}

In [9]:
columns_dict = {
    "cw-train": ['input_ids', 'attention_mask', 'labels'],
    "cw-dev": ['input_ids', 'attention_mask', 'labels'],
}   
    #"cw-dev": ['Sentence_id', 'Text', 'class_label'],
    #"test": ['id', 'rumor', 'label', 'timeline', 'evidence']
#TODO: Add shit here as well, above, not below, or perhaps below as well, who knows. 

features_dict = {}
for task_name, dataset in datasets.items():
    features_dict[task_name] = {}
    for phase, phase_dataset in dataset.items():
        features_dict[task_name][phase] = phase_dataset.map(
            convert_func_dict[task_name],
            batched=True,
            load_from_cache_file=False,
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
        #Perhaps something needs to be done here?
        #pytorch iterate dataloader TypeError: new(): invalid data type 'str'
        #https://stackoverflow.com/questions/72003569/pytorch-typeerror-new-invalid-data-type-str-when-converting-nested-list
        #https://github.com/huggingface/datasets/issues/469
        #https://stackoverflow.com/questions/77241228/typeerror-invalid-data-type-str-when-using-dataloader-to-train-scibert
        features_dict[task_name][phase].set_format(
            type='torch', 
            columns=columns_dict[task_name],
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/23 [00:00<?, ?it/s]

/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave m

cw-train train 22501 22501
cw-train train 22501 22501


/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

cw-dev train 1032 1032
cw-dev train 1032 1032


/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave m

In [10]:
from transformers import DefaultDataCollator

class NLPDataCollator(DefaultDataCollator):
    def collate_batch(self, features):
        first = features[0]
        if isinstance(first, dict):
            if "labels" in first and first["labels"] is not None:
                if first["labels"].dtype == torch.int64:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
                else:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.float)
                batch = {"labels": labels}
            for k ,v in first.items():
                if k != "labels":
                    batch[k] = torch.stack([f[k] for f in features])
            return batch
        else:
            return transformers.DefaultDataCollator().collate_batch(features)
    

class StrIgnoreDevice(str):
    def to(self, device):
        return self


In [11]:
class DataLoaderWithTaskname:
    def __init__(self, task_name, data_loader):
        self.task_name = task_name
        self.data_loader = data_loader

        self.batch_size = data_loader.batch_size
        self.dataset = data_loader.dataset

    def __len__(self):
        return len(self.data_loader)
    

    #https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
    #https://stackoverflow.com/questions/77241228/typeerror-invalid-data-type-str-when-using-dataloader-to-train-scibert
    def __iter__(self):
        for batch in self.data_loader:
            batch["task_name"] = StrIgnoreDevice(self.task_name)
            yield batch

class MultitaskDataloader:
    def __init__(self, dataloader_dict):
        self.dataloader_dict = dataloader_dict
        self.num_batches_dict = {
            task_name: len(dataloader)
            for task_name, dataloader in self.dataloader_dict.items()
        }
        self.task_name_list = list(self.dataloader_dict)
        self.dataset = [None] * sum(
            len(dataloader.dataset)
            for dataloader in self.dataloader_dict.values()
        )

    def __len__(self):
        return sum(self.num_batches_dict.values())
    
    def __iter__(self):
        task_choice_list = []
        for i, task_name in enumerate(self.task_name_list):
            task_choice_list += [i] * self.num_batches_dict[task_name]
        task_choice_list = np.array(task_choice_list)
        np.random.shuffle(task_choice_list)
        dataloader_iter_dict = {
            task_name: iter(dataloader)
            for task_name, dataloader in self.dataloader_dict.items()
        }
        for task_choice in task_choice_list:
            task_name = self.task_name_list[task_choice]
            yield next(dataloader_iter_dict[task_name])


In [12]:
from transformers import Trainer
#from transformers.data.data_collator import torch_default_data_collator
from transformers.trainer_utils import is_tf_available
from transformers.trainer import get_dataloader_sampler
#from accelerate.state import is_tf_available
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from torch.utils.data.dataloader import DataLoader
class MultitaskTrainer(Trainer):
    def get_single_train_dataloader(self, task_name, train_dataset):
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        train_sampler = RandomSampler(train_dataset)
        
        #train_sampler = (
        #    RandomSampler(train_dataset)
        #    if self.args.local_rank == -1
        #    else DistributedSampler(train_dataset) # This cucks the code up... something about missing default proc group. Prolly sumtin to do with concurrency.
        #)
        data_loader = DataLoaderWithTaskname(
            task_name=task_name,
            data_loader=DataLoader(
                train_dataset,
                batch_size=self.args.train_batch_size,
                sampler=train_sampler,
                collate_fn=self.data_collator.collate_batch,
            ),
        )

        #if is_tf_available():
        #    data_loader = nn.pl.ParallelLoader(data_loader, [self.args.device]).per_device_loader(self.args.device)
        return data_loader
    
    def get_train_dataloader(self):
        loader = MultitaskDataloader({
            task_name: self.get_single_train_dataloader(task_name, dataset)
            for task_name, dataset in self.train_dataset.items()
        })
        
        return loader
            

    

In [17]:
train_dataset = {
    task_name: dataset["train"]
    for task_name, dataset in features_dict.items()
}
trainer = MultitaskTrainer(
    model=multitask_model,
    args=transformers.TrainingArguments(
        output_dir="models/multitask_model",
        overwrite_output_dir=True,
        learning_rate=1e-5,
        do_train=True,
        num_train_epochs=6,
        per_device_train_batch_size=8,
        save_steps=5000,
    ),
    data_collator=NLPDataCollator(),
    train_dataset=train_dataset,
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/chrislindl/dat550-group/venv/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
trainer.train()

Step,Training Loss
500,0.104200
1000,0.093800
1500,0.114300
2000,0.077000
2500,0.107200
3000,0.096200
3500,0.058900
4000,0.070300
4500,0.062800
5000,0.054500


TrainOutput(global_step=17652, training_loss=0.04490024797532285, metrics={'train_runtime': 2371.9115, 'train_samples_per_second': 59.529, 'train_steps_per_second': 7.442, 'total_flos': 9352066094595072.0, 'train_loss': 0.04490024797532285, 'epoch': 6.0})